In [ ]:
## Import Necessary packages
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import wandb
import re
import os

In [ ]:
w = os.listdir("../work_dirs/mask_rcnn_r50_fpn_1x_coco")

In [ ]:
checkpts = []
for i in w:
    if i.endswith(".pth"):
        checkpts.append(i)

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
checkpts.sort(key=natural_keys)

In [ ]:
del(checkpts[-1])

In [ ]:
img_list = os.listdir("../data/coco/test2017")[:5]

## Note:
In `mmdetection/mmdet/apis/inference.py`, update the ```show_result_pyplot``` function by simply including: ```return mmcv.bgr2rgb(img)``` at the end of the function to allow **WandB** logging.


In [ ]:

config_file = '../configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'
wandb.init(project = 'ECANet-sweep')
for checkpoints in checkpts:
    log_img = []
    model = init_detector(config_file, '../work_dirs/mask_rcnn_r50_fpn_1x_coco/'+checkpoints, device='cuda:0')
    for i in img_list:
        img = '../data/coco/test2017/'+i
        result = inference_detector(model, img)
        x = show_result_pyplot(model, img, result)
        log_img.append(wandb.Image(x))
    wandb.log({"Segmentation and BBox Results": log_img})
        
wandb.run.finish()